# Example usage
## Overview
Here we will illustrate the use of the dscigametrics package to compute and analyze four key metrics within a specified time frame: New to Return Rate, Conversion Rate, Total Transaction Revenue, and Average Transaction Revenue. We'll guide you through obtaining summary statistics to grasp the market campaign overall performance, visualizing daily trends for deeper temporal insights, and identifying the most and least effective marketing campaigns based on these metrics. Whether you're tracking daily performance fluctuations or assessing the impact of marketing strategies, dscigametrics provides the tools you need to make data-driven decisions.

In [14]:
from dscigametrics.compute_metrics import compute_metrics
from dscigametrics.stat_summary import stat_summary
from dscigametrics.daily_plot import daily_plot
from dscigametrics.find_campaigns import find_campaigns
import dscigametrics
import pandas as pd

In [15]:
print(dscigametrics.__version__)

0.1.0


## Load Test Data 

In [16]:
data_path = '../tests/ga_metrics_test_data.csv'
data = pd.read_csv(data_path)

## Example of `find_campaigns` 
#### Function Description:
The `find_campaigns` function is part of a toolkit designed to analyze marketing campaign performance over a specified period using data from Google Analytics. By inputting a dataframe containing campaign information, alongside the desired date range and metrics, users can swiftly identify the most and least effective campaigns. It will return a dictionary for further use

#### Steps:
- Define the analysis period (in int or Timestamp data format)
- Prepare the list of campaign IDs to analyze (in int format)
- Decide the metric to evaluate campaign performance (in str format)
- Call function to find the best and worst campaigns based on the conversion rate

In [17]:

# Define the analysis period
start_date = 20220801 
end_date = 20220825   

campaign_ids = [219011657, 140569061, 215934049, 123851219]

# Metric to evaluate campaign performance
metric = 'conversion_rate'

# Find the best and worst campaigns based on the conversion rate
output_dict = find_campaigns(
    data=data,
    start_date=start_date,
    end_date=end_date,
    campaign_ids=campaign_ids,
    metric=metric
)



In [18]:
print(output_dict)

{'best_campaign': {'id': 123851219, 'value': 0.116}, 'worst_campaign': {'id': 219011657, 'value': 0.056}}


In [19]:

print(f"Best Campaign: ID {output_dict['best_campaign']['id']} with a {metric} of {output_dict['best_campaign']['value']}")
print(f"Worst Campaign: ID {output_dict['worst_campaign']['id']} with a {metric} of {output_dict['worst_campaign']['value']}")


Best Campaign: ID 123851219 with a conversion_rate of 0.116
Worst Campaign: ID 219011657 with a conversion_rate of 0.056
